In [23]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [85]:
is_data_loaded = True
#is_data_loaded = False

fold = '5'

In [86]:
train_src_name = 'train_fold_'+fold
test_src_name = 'test_fold_'+fold
print("Using local data")
train_data = pd.read_csv('./'+train_src_name+'.csv')



test_data = pd.read_csv('./'+test_src_name+'.csv')


alt_train_data = pd.read_csv('./'+train_src_name+'.csv')


Using local data


In [87]:
len(train_data)

26049

In [88]:
len(test_data)

6512

## Preprocess Train Data and Finally split to valid

> Add blockquote



In [89]:
# Drop missing values
train_data.dropna(inplace=True)





In [90]:
label_col = 'label'

print("Label Column: "+label_col)
# Separate features and labels
X_train_data = train_data.drop(label_col, axis=1)
y_train_data = train_data[label_col].apply(lambda x: 1 if x.strip() == '>50K' else 0)  # Convert labels to binary

# Define preprocessing for numerical and categorical columns
numerical_features = X_train_data.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train_data.select_dtypes(include=['object']).columns

Label Column: label


In [91]:
# Preprocessor (fit only on training data)
preprocessor = ColumnTransformer(transformers=[('num', StandardScaler(), numerical_features),
          ('cat', OneHotEncoder(drop='first',handle_unknown='ignore'), categorical_features)])

In [92]:
# Apply transformations
X_train_data = preprocessor.fit_transform(X_train_data)

# Convert sparse matrix to dense if necessary
if hasattr(X_train_data, 'toarray'):  # Check if X is sparse
    X_train_data = X_train_data.toarray()


print("Train set transformed successfully!")

X_train = X_train_data
y_train = y_train_data

Train set transformed successfully!


In [93]:
len(y_train_data)

26049

In [94]:
label_col

'label'

## Preprocess Test Data

In [95]:
# Now apply the same transformations to the test set
test_data.dropna(inplace=True)

# Separate features and labels for the test set
X_test_data = test_data.drop(label_col, axis=1)
y_test_data = test_data[label_col].apply(lambda x: 1 if x.strip() == '>50K' else 0)  # Convert labels to binary

# Transform the test data using the already fitted preprocessor
X_test_data = preprocessor.transform(X_test_data)

# Convert sparse matrix to dense if necessary for the test set
if hasattr(X_test_data, 'toarray'):
    X_test_data = X_test_data.toarray()

# Now you can use X_test and y_test in your model evaluation or testing
print("Test set transformed successfully!")

X_test = X_test_data
y_test = y_test_data

Test set transformed successfully!


## Gradient Boosted Classifier

In [96]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# Initialize the classifier
gbc = GradientBoostingClassifier()

# Train the model
gbc.fit(X_train, y_train)

# Predict on the test set
y_pred = gbc.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.8688574938574939
              precision    recall  f1-score   support

           0       0.88      0.95      0.92      4944
           1       0.80      0.61      0.69      1568

    accuracy                           0.87      6512
   macro avg       0.84      0.78      0.80      6512
weighted avg       0.86      0.87      0.86      6512

Confusion Matrix:
[[4703  241]
 [ 613  955]]


# Save into a new dataframe

In [97]:
def evaluate_and_save_gbc(model, X_train, original_train_data, output_file='train_predictions_gbc.csv'):

  # Get the model's predictions (using probability threshold of 0.5 for binary classification)
  y_pred = model.predict(X_train)


  # Convert binary predictions to the original format ('>50K' and '<=50K')
  y_pred_str = ['>50K' if y == 1 else '<=50K' for y in y_pred]

  # Add predictions to the original DataFrame
  original_train_data['predictions'] = y_pred_str

  # Save the combined DataFrame with predictions to a CSV file
  original_train_data.to_csv(output_file, index=False)

  print(f"Predictions saved to {output_file}")

In [98]:
evaluate_and_save_gbc(gbc, X_train, train_data, output_file='pred_'+test_src_name+'.csv')

Predictions saved to pred_test_fold_5.csv


In [99]:
# Example usage:
# original_test_data is the DataFrame containing the original test data (with features, but without labels)
# test_loader is the DataLoader for the test set
evaluate_and_save_gbc(gbc, X_test, test_data, output_file='pred_'+test_src_name+'.csv')

Predictions saved to pred_test_fold_5.csv
